In [2]:
import smtplib
import schedule
import time
import requests
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

def get_exchange_rate():
    response = requests.get('https://api.exchangeratesapi.io/latest?base=USD')
    data = response.json()
    return data['rates']['INR']

def send_email(rate):
    msg = MIMEMultipart()
    msg['From'] = 'your-email@gmail.com'
    msg['To'] = 'your-email@gmail.com'
    msg['Subject'] = 'Current USD to INR Exchange Rate'
    body = f'The current exchange rate from USD to INR is {rate}.'
    msg.attach(MIMEText(body, 'plain'))

    server = smtplib.SMTP('smtp.gmail.com', 587)
    server.starttls()  # Secure the connection
    server.login(msg['From'], 'your-password')  # Login with your email and password

    server.send_message(msg)
    server.quit()

def job():
    rate = get_exchange_rate()
    send_email(rate)

schedule.every().monday.at("08:00").do(job)
schedule.every().tuesday.at("08:00").do(job)
schedule.every().wednesday.at("08:00").do(job)
schedule.every().thursday.at("08:00").do(job)
schedule.every().friday.at("08:00").do(job)

while True:
    schedule.run_pending()
    time.sleep(1)


KeyboardInterrupt: 